# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [6]:
import os
from glob import glob

PRICE_DATA = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive= True)

parquet_files



['../../05_src/data/prices/SB/SB_2015/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2015/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2012/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2012/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2013/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2013/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2014/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2014/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2009/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2009/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2008/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2008/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2011/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2011/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2016/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2016/part.1.parquet',
 '../../05_src/data/prices/SB/SB_2020/part.0.parquet',
 '../../05_src/data/prices/SB/SB_2020/part.1.parquet',
 '../../05

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
# Write your code below.
dd_price = dd.read_parquet(parquet_files).set_index('ticker')

dd_lags = dd_price.groupby('ticker', group_keys = False).apply(
    lambda x:x.assign(Close_lag_1 = x['Close'].shift(1),
                      Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

dd_rets = dd_lags.assign(Returns = lambda x:
    x['Close']/x['Close_lag_1']-1)

dd_feat = dd_rets.assign(hi_lo_range = lambda x:
    x['High'] - x['Low'])

dd_feat


/var/folders/qh/jt17nfyx34g5ypbk6rvwjr_my2784y/T/ipykernel_97884/2146570995.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lags = dd_price.groupby('ticker', group_keys = False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
npartitions=89,,,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
ACB,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTO,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTO,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [8]:
# Write your code below.
pd_feat = dd_feat.compute()
pd_feat = pd_feat.sort_values(['ticker', 'Date'], ascending=[True, False])
pd_final = pd_feat.assign(moving_avg = pd_feat.groupby('ticker')['Returns'].rolling(10).mean().reset_index(level=0, drop = True))
print(pd_final)


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2020-04-01  69.470001  70.230003  68.150002  68.919998  68.919998   
A      2020-03-31  72.339996  72.800003  70.500000  71.620003  71.620003   
A      2020-03-30  71.059998  73.180000  71.059998  72.669998  72.669998   
A      2020-03-27  71.550003  73.209999  70.279999  70.910004  70.730003   
A      2020-03-26  70.000000  74.449997  69.650002  73.720001  73.532867   
...           ...        ...        ...        ...        ...        ...   
ZTO    2016-11-02  16.330000  16.389999  15.820000  16.000000  15.578986   
ZTO    2016-11-01  17.000000  17.049999  15.545000  16.000000  15.578986   
ZTO    2016-10-31  17.190001  17.190001  16.850000  16.930000  16.484514   
ZTO    2016-10-28  16.790001  17.250000  16.680000  16.990000  16.542934   
ZTO    2016-10-27  18.400000  18.450001  16.500000  16.570000  16.133986   

           

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ No, we have Dask
+ Would it have been better to do it in Dask? Why?
+ Not in this instance, as data set is not that large. We should use Dask if dataset is huge.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.